In [1]:
# Importar librerías 

import pandas as pd  

from bs4 import BeautifulSoup
import requests

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from time import sleep  

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import re


In [1]:
class ExtraerDatosPeliculasSelenium:
    def __init__(self, archivo_csv): 
        self.archivo_csv = pd.read_csv(archivo_csv)  
        self.info_peliculas= []
        self.info_peliculas_tuplas = []
        self.info_actores = []
    

    def pag_principal_google(self):
        # Inicializar el driver de Chrome
        options = webdriver.ChromeOptions()
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        self.driver = webdriver.Chrome(options=options)
        sleep(2)
        self.driver.maximize_window()
        self.driver.get("https://www.google.es/")
        # acepta cookies
        sleep(5)
        try:
            self.driver.find_element("css selector", "#L2AGLb").click()
        except:
            pass
        self.escoger_pelicula_csv()         

    def escoger_pelicula_csv (self):   
          
        # Itera por cada pelicula del csv
        for peli in range(0, len(self.archivo_csv)):  
            # Escoge info del csv para la busqueda        
            id_pelicula = self.archivo_csv["Id"][peli]          
            pelicula_anho = (f"{self.archivo_csv['Titulo'][peli]} {self.archivo_csv['Año'][peli]}")  
            # Mete en la lista el id     
            self.info_peliculas.append(id_pelicula)              
            self.busqueda_google("imdb", id_pelicula)
            self.busqueda_google("rottentomatoes", pelicula_anho)
            # Comprueba que la lista peliculas tenga mas de un dato (evitar que solo tenga el dato de tomatoes) lo pasa a la lista de tuplas y resetea el contenido           
            if len(self.info_peliculas) > 1:
                tupla = tuple(self.info_peliculas)
                self.info_peliculas_tuplas.append(tupla)
            self.info_peliculas.clear()
            
        self.guardar_info_peliculas_csv(f"info_peliculas_{self.archivo_csv['Genero'][peli]}")
       
    def busqueda_google(self, pagina, dato_buscador):
        sleep(3)
        # Ingresar el título y año de la película en el campo de búsqueda
        self.driver.find_element("css selector", "#APjFqb").clear()
        self.driver.find_element("css selector", "#APjFqb").send_keys(f'{pagina} "{dato_buscador}"', Keys.ENTER)              

        # Definimos la url de la página de la que vamos a sacar datos y llama a la api   
        try:            
            sleep(5)            
            href = self.driver.find_element("css selector", "#rso .yuRUbf > div > span > a").get_attribute("href")   
            # si el primer resultado de google no es de imdb pasa a la siguiente pelicula 
            if pagina in href:                
                self.llamada_api(href, pagina) 
            else:
                print(f"ha fallado la llamada a la pagina {pagina}")
                if "rotten" in pagina:
                    self.info_peliculas.append("")
        # si no consigue la url de google de tomatoes añade none (esa pelicula no esta en rotten pero si en imdb)
        except:
            print("No consigo la url de google")
            if "rotten" in pagina:
                    self.info_peliculas.append("")
            pass     
            sleep(5)    
    
    def llamada_api (self, href, headers):

        # Definimos un User-Agent para que la solicitud parezca venir de un navegador real
        # Elegimos el headers en funcion de si vamos a llamar a imdb o rotten
        if "imdb" in headers:
            headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }
        elif "rotten" in headers:
            headers = ""        

        # Hacemos la request a la página de la que queremos sacar la info
        resultado = requests.get(href, headers=headers)

        # Vemos si todo ha ido bien
        print("La respuesta de la petición es:", resultado.status_code)

        # creamos el objeto BeautifulSoup para poder acceder al contenido solicitado
        sopa = BeautifulSoup(resultado.content, 'html.parser')
        # si el objeto viene de imdb llamo a su método para limpiar los datos
        if resultado.status_code == 200:    
            if headers: 
                self.limpieza_datos_peliculas_imdb(sopa)  
        # si el objeto viene de rotten llamo a su método para limpiar los datos
            else:
                self.limpieza_datos_peliculas_tomatoes(sopa) 
        else:
            return 
    
    def limpieza_datos_peliculas_imdb(self, sopa):
        # sacamos info        
        puntuacion_imdb = sopa.find("span", {"class": "sc-eb51e184-1 cxhhrI"}).text        
        # puntiacion
        if puntuacion_imdb:
            self.info_peliculas.append(puntuacion_imdb)
        else:
            self.info_peliculas.append("")
        # argumento
        argumento = sopa.find("span", {"class": "sc-2d37a7c7-0"}).text
        if argumento:            
            self.info_peliculas.append(argumento)
        else:
            self.info_peliculas.append("") 
        # titulo
        titulo_imbd = sopa.find("h1", {"class": "sc-d8941411-0"}).text  
        if titulo_imbd:
            self.info_peliculas.append(titulo_imbd)
        else:
            self.info_peliculas.append("")  
        # director/guionista
        director_encontrado = False
        guionista_encontrado = False
        li = sopa.find_all("li", {"class": "ipc-metadata-list__item"})

        for i in li[0:3]:            
            if "Directors" in i.text:
                director = i.text.replace("Directors","")                          
                self.info_peliculas.append(director)     
                director_encontrado = True

            elif "Director" in i.text:
                director = i.text.replace("Director","")               
                self.info_peliculas.append(director)     
                director_encontrado = True           

            elif "Writers" in i.text:                
                guionista = i.text.replace("Writers","")             
                self.info_peliculas.append(guionista)
                guionista_encontrado = True

            elif "Writer" in i.text:
                guionista = i.text.replace("Writer","")            
                self.info_peliculas.append(guionista)                
                guionista_encontrado = True

        if not director_encontrado:
            self.info_peliculas.append("") 
        if not guionista_encontrado:
            self.info_peliculas.append("") 

        # duracion
        duracion_encontrada = False
        li2 = sopa.find_all("li", {"class": "ipc-inline-list__item"})
        for i in li2[0:11]:
            if re.search(r'\d+h\s?\d*m?', i.text):
                duracion = i.text
                self.info_peliculas.append(duracion)
                duracion_encontrada = True
        if not duracion_encontrada:
            self.info_peliculas.append("")
        # actores
        try:
            actores = sopa.find_all("a", {"class": "sc-bfec09a1-1 gCQkeh"})            
            for i in range(0,10):   
                actor = actores[i].text
                # apendiamos el id de la pelicula por cada actor
                self.info_actores.append(self.info_peliculas[0])
                self.info_actores.append(actor)                
        except:
            pass

    def limpieza_datos_peliculas_tomatoes (self, sopa):
        # sacamos info de la sopa
        div = sopa.find_all("div", {"class": "media-scorecard"})
        puntuacion_tomatoes = div[0].find("rt-text").text
        # puntiacion
        if puntuacion_tomatoes:
            self.info_peliculas.append(puntuacion_tomatoes)
        else:
            self.info_peliculas.append("")       

    def guardar_info_peliculas_csv (self, nombre_archivo):
        df = pd.DataFrame(self.info_peliculas_tuplas, columns=["id", "puntuacion_imdb", "argumento", "titulo", "director", "guionista", "duracion", "puntuacion_tomatoes"])
        df.to_csv(nombre_archivo+".csv", index=False, header=True)
            
       

In [2]:
# csv = "action.csv"
csv = input("Introduce el género del que quieres conseguir la información")
try:
    peliculas_accion_sele = ExtraerDatosPeliculasSelenium(csv+".csv")
    peliculas_accion_sele.pag_principal_google()
except:
    print("Hay algún error, comprueba que exista el fichero")


Hay algún error, comprueba que exista el fichero


In [5]:
archivo_csv = pd.read_csv("drama.csv")  
archivo_csv

,Tipo,Titulo,Mes,Año,Id,Genero
0,Movie,Pál Adrienn,6.0,2011.0,tt0146592,Drama
1,Movie,Oda az igazság,2.0,2010.0,tt0154039,Drama
2,Movie,A zöld sárkány gyermekei,4.0,2011.0,tt0162942,Drama
3,Movie,The Final Journey,9.0,2010.0,tt0230212,Drama
4,Movie,In My Sleep,4.0,2010.0,tt0326965,Drama
...,...,...,...,...,...,...
237,Movie,Mom's Coming,11.0,2024.0,tt10579986,Drama
238,Movie,Love Song and Power,1.0,2024.0,tt10597228,Drama
239,Movie,Spaceman,3.0,2024.0,tt11097384,Drama
240,Movie,IF,5.0,2024.0,tt11152168,Drama


: 